In [1]:
'''
to check if using semantic closeness we can get classification
This can later be used to train the betr with lableled data and become a better classifier
'''

'\nto check if using semantic closeness we can get classification\nThis can later be used to train the betr with lableled data and become a better classifier\n'

In [2]:
# using normal bert

In [3]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from pandas import DataFrame

In [4]:
# Load the pre-trained Sentence Transformer model
model = SentenceTransformer('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
# # Define some example queries and documents
# documents = ["hab", "govt department"]
# queries = [
#     "My tata nano was not starting after service, when complained they ignored me",
#     "The department of ayush has not responded on the recent scandal"
# ]

# # Encode the queries and documents using the LLM into embeddings
# query_embeddings = model.encode(queries)
# document_embeddings = model.encode(documents)

In [6]:
%%time
# Define some example queries and documents
actual_tag = {"AE":["pain", "side effect", "bodily harm", "suffer", "physical discomfort"], "PC" : ["device related issue","malfunctioning device","broken device", "mechanical failure"]}
documents = ["pain", "side effect", "bodily harm", "suffer", "physical discomfort", "device related issue","malfunctioning device","broken device", "mechanical failure"]
queries = [
    "My eye is red after taking paracetamol and is burning",
    "The syringe of dasfarut injection kit was bent",
    "it was a usual day. I went to the office and then went home"
]

# Encode the queries and documents using the LLM into embeddings
query_embeddings = model.encode(queries)
document_embeddings = model.encode(documents)

CPU times: user 1.01 s, sys: 141 ms, total: 1.15 s
Wall time: 1.45 s


In [7]:
%%time
# Iterate through each query embedding and compare
# it with all document embeddings using cosine similarity.
# Cosine similarity measures how similar two vectors are directionally.
import numpy as np
output = {
    "input" :[],
    "associated_tags" : [],
    "associated_score" : []
}

for i, query_embedding in enumerate(query_embeddings):
  max_similarity = -1
  closest_document = None
  for j, document_embedding in enumerate(document_embeddings):
    similarity = np.dot(query_embedding, document_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(document_embedding))
    if similarity > max_similarity:
      max_similarity = similarity
      closest_document = documents[j]

  output["input"].append(queries[i])
  output["associated_tags"].append(closest_document)
  output["associated_score"].append(max_similarity)
output
df = DataFrame(output)
df

CPU times: user 1.88 ms, sys: 31 µs, total: 1.91 ms
Wall time: 1.49 ms


,input,associated_tags,associated_score
0,My eye is red after taking paracetamol and is ...,pain,0.635839
1,The syringe of dasfarut injection kit was bent,device related issue,0.627603
2,it was a usual day. I went to the office and t...,device related issue,0.440397


In [8]:
df["ae_pc"] = df["associated_tags"].apply(lambda x : [key for key, value in actual_tag.items() if x in value ][0])
df

,input,associated_tags,associated_score,ae_pc
0,My eye is red after taking paracetamol and is ...,pain,0.635839,AE
1,The syringe of dasfarut injection kit was bent,device related issue,0.627603,PC
2,it was a usual day. I went to the office and t...,device related issue,0.440397,PC


In [9]:
%%time
# get sentiement
sentiment = []
model_s = 'nlptown/bert-base-multilingual-uncased-sentiment'

# Load the pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_s)
model = AutoModelForSequenceClassification.from_pretrained(model_s)

# Tokenize the text
for i in queries:
  inputs = tokenizer(i, return_tensors="pt")

  # Get the sentiment score
  outputs = model(**inputs)
  scores = outputs[0][0].detach().numpy()
  sentiment.append(scores[0] - scores[1])
df["sentiment"] = sentiment
df


CPU times: user 594 ms, sys: 57.8 ms, total: 651 ms
Wall time: 899 ms


,input,associated_tags,associated_score,ae_pc,sentiment
0,My eye is red after taking paracetamol and is ...,pain,0.635839,AE,0.537153
1,The syringe of dasfarut injection kit was bent,device related issue,0.627603,PC,-0.295873
2,it was a usual day. I went to the office and t...,device related issue,0.440397,PC,-0.020665


In [10]:
final_df = df[(df["sentiment"]>0) | (df["associated_score"]>0.5)]
final_df

,input,associated_tags,associated_score,ae_pc,sentiment
0,My eye is red after taking paracetamol and is ...,pain,0.635839,AE,0.537153
1,The syringe of dasfarut injection kit was bent,device related issue,0.627603,PC,-0.295873


In [11]:
# modern bert

In [13]:
# model_2 = SentenceTransformer('answerdotai/ModernBERT-base')